## Gaia

https://gea.esac.esa.int/archive/

In [ ]:
from bioexplorer import BioExplorer, Vector3

In [ ]:
# !pip install astropy

In [ ]:
import requests
import gzip
import astropy.coordinates as coord
import astropy.units as u
from tqdm import tqdm
import os


url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
status = be.reset_scene()

# Specify the URL of the file to be downloaded
url = "http://cdn.gea.esac.esa.int/Gaia/gdr1/gaia_source/csv/"
cache_folder = '/home/favreau/data/gaia'

status = be.reset_scene()
for a in range(256):
    for b in range(256):
        for c in range(256):
            positions = list()
            radii = list()

            # Download the file
            base_name = 'GaiaSource_%03d-%03d-%03d' % (a, b, c)
            path = os.path.join(cache_folder, base_name + '.csv.gz')
            lines = list()
            if not os.path.exists(path):
                response = requests.get(url + base_name + '.csv.gz')
                content = response.content

                # Save the file
                with open(path, 'wb') as file:
                    file.write(content)
                lines = gzip.decompress(content).decode('utf-8').split('\n')
            else:
                lines = gzip.open(path).readlines()

            header = True
            for line in tqdm(lines):
                if header:
                    header = False
                    continue
                values = str(line).split(',')

                # Convert the RA and DEC coordinates to 3D coordinates
                ra = float(values[4])
                dec = float(values[6])
                icrs = coord.SkyCoord(ra=ra * u.degree, dec=dec * u.degree, frame='icrs')
                galactic = icrs.transform_to(coord.Galactic)

                # Extract the x, y, z coordinates
                x = float(galactic.cartesian.x)
                y = float(galactic.cartesian.y)
                z = float(galactic.cartesian.z)
                
                positions.append(Vector3(x, y, z))
                radii.append(0.00005)

            status = be.add_spheres(
                name=base_name,
                positions=positions, radii=radii)
